# Exploratory Data Analysis - Valorant

## Libs

### Data manipulation

In [1]:
import pandas as pd
from datetime import datetime
import pathlib

### Charts

In [2]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
import warnings
warnings.filterwarnings("ignore")

## Initial Settings and Loads

### Pathlib

In [3]:
abs_path = pathlib.Path.cwd().parent

### Load Players Rank

In [4]:
file_read = pathlib.Path(abs_path).joinpath('data').joinpath('players').joinpath('players_processed.csv')
df_players_raw = pd.read_csv(file_read)
del df_players_raw['Unnamed: 0']

### Load Matches

In [5]:
file_read = pathlib.Path(abs_path).joinpath('data').joinpath('matches').joinpath('matches_processed.csv')
df_matches_raw = pd.read_csv(file_read)
del df_matches_raw['Unnamed: 0']

### Load Weapons

In [6]:
file_read = pathlib.Path(abs_path).joinpath('data').joinpath('weapons').joinpath('weapons_processed.csv')
df_weapons_raw = pd.read_csv(file_read)
del df_weapons_raw['Unnamed: 0']

### Selected Players

In [7]:
selected_players_list = (
    df_players_raw
    .sort_values('rankedRating', ascending = False)
    .reset_index(drop = True)
    .head(10)
    )
selected_players_list = selected_players_list['userId'].to_list()
selected_players_list

['neveR%23god',
 'RieNs%23wagwn',
 'SEN zekken%235193',
 'SR mada%231945',
 'TBK anao sato%23lukxo',
 'artzin%23psz',
 'Heretics AvovA%23uwu',
 'TBG lurzyyyy%23Boss',
 'TSM NaturE%23RAT',
 'KC TakaS%23KLWM']

## EDA Players

### Ranking

In [8]:
players_ranking = (
    df_players_raw[['userId', 'server', 'leaderboardRank', 'rankedRating', 'avgScore', 'competitiveTier']]
    .sort_values('rankedRating', ascending = False)
    .reset_index(drop = True)
    .head(10)
    )

In [9]:
fig = px.histogram(players_ranking, x='userId', y='rankedRating', height=400)
fig.show()

In [10]:
fig = px.histogram(players_ranking, x='userId', y='avgScore', height=400)
fig.show()

In [11]:
players_ranking.sort_values(['leaderboardRank', 'rankedRating'], ascending = True)

,userId,server,leaderboardRank,rankedRating,avgScore,competitiveTier
4,TBK anao sato%23lukxo,br,1,1127,249.0,27
2,SEN zekken%235193,na,1,1159,268.0,27
0,neveR%23god,eu,1,1252,275.0,27
5,artzin%23psz,br,2,1122,261.0,27
3,SR mada%231945,na,2,1147,249.0,27
1,RieNs%23wagwn,eu,2,1183,266.0,27
7,TBG lurzyyyy%23Boss,eu,3,1119,245.0,27
8,TSM NaturE%23RAT,na,3,1119,266.0,27
6,Heretics AvovA%23uwu,eu,4,1119,251.0,27
9,KC TakaS%23KLWM,eu,5,1107,268.0,27


### Matches

In [12]:
matches_per_player = (
                        df_matches_raw
                        .query('userId == @selected_players_list')
                        .groupby(['userId'])['userId']
                        .count()
                        .reset_index(name = 'count')
                        .reset_index(drop = True)
                        )

In [13]:
fig = px.line(matches_per_player, x = 'userId', y = 'count', title = 'Number of matches per player')
fig.show()

In [14]:
fig = px.box(matches_per_player, y = 'count', title = 'Matches in intervals between quartiles')
fig.show()

### Time

In [15]:
total_time_player = (
    df_matches_raw
    .query('userId == @selected_players_list')
    .groupby(['userId'])[['playtimeValue']]
    .sum()
    .reset_index()
)
total_time_player['playtimeValueMinutes'] = (df_matches_raw['playtimeValue']/60).round(2)

In [16]:
fig = px.bar(total_time_player, x = 'userId', y = 'playtimeValueMinutes', color = 'playtimeValueMinutes', title = 'Means time per days of week')
fig.show()

In [17]:
fig = px.box(total_time_player, y="playtimeValueMinutes", title = 'Time in intervals between quartiles')
fig.show()

### Results

In [18]:
number_wins_player = (
                        df_matches_raw
                        .query('userId == @selected_players_list')
                        .groupby(['userId', 'result'])['result']
                        .count()
                        .reset_index(name = 'count')
                        .sort_values(['count'], ascending = False)
                        .reset_index(drop = True)
                        )

In [19]:
fig = go.Figure()
fig.add_trace(go.Scatter(y=number_wins_player.query('result == "victory"')['count'],
                    name='Victory'))
fig.add_trace(go.Scatter(y=number_wins_player.query('result == "defeat"')['count'],
                    name='Defeat'))
fig.add_trace(go.Scatter(y=number_wins_player.query('result == "tied"')['count'],
                    name='Tied'))                    

fig.show();

In [20]:
fig = px.box(number_wins_player, x = "result", y = "count", color = "result")
fig.update_traces(quartilemethod="exclusive") # or "inclusive", or "linear" by default
fig.show()

### Headshots

In [21]:
number_headshot_per_player = (
    df_matches_raw
    .query('userId == @selected_players_list')
    .groupby(['userId'])['headshotsValue']
    .sum()
    .reset_index(name = 'sum')
    .sort_values('sum', ascending=False)
    .reset_index(drop = True)
)

In [22]:
fig = px.line(number_headshot_per_player, x = 'userId', y = 'sum', title = 'Means time per days of week')
fig.show()

In [23]:
fig = px.box(number_headshot_per_player, y="sum", title = 'Interval of time played between quartiles by players')
fig.show()

### KDA

In [24]:
number_kda_player = (
                        df_matches_raw
                        .query('userId == @selected_players_list')
                        .groupby('userId')[['killsValue', 'deathsValue', 'assistsValue','userId']]
                        .sum()
                        .sort_values(['killsValue', 'deathsValue', 'assistsValue'], ascending = False)
                        .reset_index()
                        )

In [25]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=number_kda_player['userId'], y=number_kda_player['killsValue'],
                    mode='lines',
                    name='killsValue'))
fig.add_trace(go.Scatter(x=number_kda_player['userId'], y=number_kda_player['deathsValue'],
                    mode='lines',
                    name='deathsValue'))
fig.add_trace(go.Scatter(x=number_kda_player['userId'], y=number_kda_player['assistsValue'],
                    mode='lines',
                    name='assistsValue'))

fig.show()

In [26]:
number_kda_player = number_kda_player.drop(columns = 'userId')

fig = go.Figure()

for col in number_kda_player:
  fig.add_trace(go.Box(y=number_kda_player[col].values, name=number_kda_player[col].name))
  
fig.show()

### Damage

In [27]:
number_damage_player = (
    df_matches_raw
    .query('userId == @selected_players_list')
    .groupby(['userId'])['damageValue','damageReceivedValue']
    .sum()
    .reset_index()
)

In [28]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=number_damage_player['userId'], y=number_damage_player['damageValue'],
                    mode='lines',
                    name='damageValue'))
fig.add_trace(go.Scatter(x=number_damage_player['userId'], y=number_damage_player['damageReceivedValue'],
                    mode='lines',
                    name='damageReceivedValue'))
fig.show()

### Economy

In [29]:
number_economy_player = (
    df_matches_raw
    .query('userId == @selected_players_list')
    .groupby(['userId'])['econRatingValue']
    .mean()
    .sort_values(ascending = False)
    .reset_index()
)

In [30]:
fig = go.Figure()
fig.add_trace(go.Box(y=number_economy_player['econRatingValue'], name = 'econRatingValue' ))
fig.show()

In [31]:
fig = go.Figure()
fig.add_trace(go.Bar(x=number_economy_player['userId'], y=number_economy_player['econRatingValue'],
                    name='econRatingValue'))
fig.show()

### Weapons

In [76]:
# from IPython.display import display
# with pd.option_context('display.max_rows', 180, 'display.max_columns', 25):
#     display(df_weapons_raw.query('userId == @selected_players_list').reset_index(drop = True)).max_rows()

In [79]:
df_weapons_raw.head(5)

,userId,weaponName,matchesPlayedValue,matchesWonValue,matchesLostValue,matchesTiedValue,matchesWinPctValue,roundsPlayedValue,killsValue,killsPerRoundValue,...,damageValue,damagePerRoundValue,damagePerMatchValue,damageReceivedValue,dealtHeadshotsValue,dealtBodyshotsValue,dealtLegshotsValue,killDistanceValue,avgKillDistanceValue,longestKillDistanceValue
0,TBK anao sato%23lukxo,marshal,565,289,259,17,51.150442,234,780,0.628205,...,128466,92.611111,227.373451,0,439,412,3,413923,732.607080,6837
1,TBK anao sato%23lukxo,classic,1195,608,561,26,50.878661,2012,1192,0.133698,...,137827,15.215209,115.336402,0,1244,1504,81,346357,289.838494,4575
2,TBK anao sato%23lukxo,phantom,1276,609,637,30,47.727273,916,5017,0.899563,...,805722,148.398472,631.443574,0,4221,5949,392,1483714,1162.785266,5426
3,TBK anao sato%23lukxo,shorty,122,70,50,2,57.377049,40,93,0.625000,...,10300,65.275000,84.426230,0,118,604,47,12410,101.721311,735
4,TBK anao sato%23lukxo,ares,40,27,13,0,67.500000,2,91,1.000000,...,11427,81.000000,285.675000,0,62,224,45,5082,127.050000,2639


#### Most used weapons

In [59]:
df_weapons_most_used = (
    df_weapons_raw
    .query('userId == @selected_players_list')
    .groupby('userId')['weaponName', 'roundsPlayedValue','matchesPlayedValue']
    .max()
    .reset_index()
    .sort_values('roundsPlayedValue', ascending = False)

)
df_weapons_most_used

,userId,weaponName,roundsPlayedValue,matchesPlayedValue
5,TBG lurzyyyy%23Boss,vandal,15919,5561
9,neveR%23god,vandal,8733,2582
3,SEN zekken%235193,vandal,8632,2132
4,SR mada%231945,vandal,8382,2575
2,RieNs%23wagwn,vandal,8215,3845
6,TBK anao sato%23lukxo,vandal,7458,2426
8,artzin%23psz,vandal,6773,3261
0,Heretics AvovA%23uwu,vandal,6749,1773
7,TSM NaturE%23RAT,vandal,5253,2105
1,KC TakaS%23KLWM,vandal,4487,1075


In [58]:
fig = go.Figure()

fig.add_trace(go.Bar(x=df_weapons_most_used['userId'], y=df_weapons_most_used['matchesPlayedValue'],
                    name='matchesPlayedValue', text='Vandal'))
fig.add_trace(go.Bar(x=df_weapons_most_used['userId'], y=df_weapons_most_used['roundsPlayedValue'],
                    name='roundsPlayedValue', text='Vandal'))
fig.update_layout(title = 'Most used weapon')

fig.show()

#### Results by weapons

In [64]:
df_weapons_most_used = (
    df_weapons_raw
    .query('userId == @selected_players_list')
    .groupby(['userId'])[['weaponName','matchesWonValue', 'matchesLostValue', 'matchesTiedValue']]
    .max()
    .sort_values('matchesWonValue', ascending=False)
    .reset_index()
)
df_weapons_most_used

,userId,weaponName,matchesWonValue,matchesLostValue,matchesTiedValue
0,TBG lurzyyyy%23Boss,vandal,2881,2611,69
1,RieNs%23wagwn,vandal,2041,1750,54
2,artzin%23psz,vandal,1697,1494,70
3,neveR%23god,vandal,1440,1104,40
4,SR mada%231945,vandal,1439,1108,28
5,SEN zekken%235193,vandal,1286,831,15
6,TBK anao sato%23lukxo,vandal,1214,1166,46
7,TSM NaturE%23RAT,vandal,1132,950,23
8,Heretics AvovA%23uwu,vandal,1038,709,26
9,KC TakaS%23KLWM,vandal,657,402,16


In [70]:
df_weapons_most_used = (
    df_weapons_raw
    .query('userId == @selected_players_list')
    .groupby(['userId'])[['killsValue', 'killsPerRoundValue']]
    .sum()
    .sort_values('killsValue', ascending=False)
    .reset_index()
)
df_weapons_most_used

,userId,killsValue,killsPerRoundValue
0,TBG lurzyyyy%23Boss,100033,11.364133
1,RieNs%23wagwn,70020,12.880045
2,artzin%23psz,57882,73.910453
3,neveR%23god,49058,51.154913
4,SR mada%231945,47983,11.789442
5,TBK anao sato%23lukxo,44437,12.808882
6,SEN zekken%235193,43261,13.230447
7,TSM NaturE%23RAT,40182,12.333626
8,Heretics AvovA%23uwu,39157,20.402591
9,KC TakaS%23KLWM,20901,28.669701


In [83]:
fig = go.Figure()

fig.add_trace(go.Bar(x=df_weapons_most_used['userId'], y=df_weapons_most_used['killsValue'],
                    name='killsValue', text='Vandal'))

fig.update_layout(title = 'Kills by weapons')

fig.show()

In [84]:
fig = go.Figure()
fig.add_trace(go.Bar(x=df_weapons_most_used['userId'], y=df_weapons_most_used['killsPerRoundValue'],
                    name='killsPerRoundValue', text='Vandal'))
fig.update_layout(title = 'Kills by weapons')
fig.show()

#### Shoot stats by weapons

In [87]:
df_weapons_most_used = (
    df_weapons_raw
    .query('userId == @selected_players_list')
    .groupby(['userId'])[['dealtHeadshotsValue', 'dealtBodyshotsValue', 'dealtLegshotsValue']]
    .sum()
    .sort_values('dealtHeadshotsValue', ascending=False)
    .reset_index()
)
df_weapons_most_used

,userId,dealtHeadshotsValue,dealtBodyshotsValue,dealtLegshotsValue
0,TBG lurzyyyy%23Boss,82271,90386,6090
1,RieNs%23wagwn,51061,82895,5312
2,SR mada%231945,37940,52411,4238
3,artzin%23psz,37739,81351,5447
4,TBK anao sato%23lukxo,35261,42922,2354
5,SEN zekken%235193,33006,51105,4093
6,Heretics AvovA%23uwu,30464,46786,3354
7,neveR%23god,30114,55290,2846
8,TSM NaturE%23RAT,28954,51940,3413
9,KC TakaS%23KLWM,13915,26794,1629


In [88]:
fig = go.Figure()
fig.add_trace(go.Bar(x=df_weapons_most_used['userId'], y=df_weapons_most_used['dealtHeadshotsValue'],
                    name='dealtHeadshotsValue', text='Vandal'))
fig.add_trace(go.Bar(x=df_weapons_most_used['userId'], y=df_weapons_most_used['dealtBodyshotsValue'],
                    name='dealtBodyshotsValue', text='Vandal'))
fig.add_trace(go.Bar(x=df_weapons_most_used['userId'], y=df_weapons_most_used['dealtLegshotsValue'],
                    name='dealtLegshotsValue', text='Vandal'))
fig.update_layout(title = 'Shoot stats by weapons')
fig.show()

## EDA Matches

### Days

In [ ]:
matches_per_day = (
    
                    df_matches_raw
                    .query("userId == @selected_players_list")
                    .groupby([df_matches_raw.dateMatch, df_matches_raw.weekDay])['matchId']
                    .count()
                    .reset_index(name='count')
                    .reset_index(drop = True)
                    )

In [ ]:
fig = px.line(x = matches_per_day['dateMatch'], y = matches_per_day['count'], title= 'Number of matches per day')
fig.show()

In [ ]:
fig = px.line(x = matches_per_day['dateMatch'], y = matches_per_day['count'], title= 'Number of matches per day')
fig.show()

In [ ]:
weekDayOrdered = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
mean_matches_per_days_week = (
                                matches_per_day.groupby(matches_per_day.weekDay)['count']
                                .mean()
                                .reindex(weekDayOrdered) 
                                .reset_index(name = 'mean')
                                .reset_index(drop = True)
                                .round(3)
                                )

In [ ]:
fig = px.line(mean_matches_per_days_week, x = 'weekDay', y = 'mean', title = 'Means of matches per days of week')
fig.show()

In [ ]:
means_time_matches = df_matches_raw[['matchId', 'weekDay', 'playtimeValue']]
means_time_matches['playtimeValueMinutes'] = means_time_matches.playtimeValue/60

means_time_matches = (
                        means_time_matches.groupby('weekDay')['playtimeValueMinutes']
                        .mean()
                        .reindex(weekDayOrdered)
                        .reset_index(name = 'mean')
                        .reset_index(drop = True)                     
                        .round(2)
)

In [ ]:
fig = px.line(means_time_matches, x = 'weekDay', y = 'mean', title = 'Means time per days of week')
fig.show()

In [ ]:
top_seven_days_matches = (
                            df_matches_raw.groupby(['dateMatch', 'weekDay'])['weekDay']
                            .count()
                            .reset_index(name = 'count')
                            .reset_index(drop = True)
                            .sort_values('count', ascending = False)
                            .head(7)
                            )

In [ ]:
fig = px.bar(top_seven_days_matches, x='weekDay', y='count',
             hover_data=['dateMatch', 'count'], color='count', height=400, title = 'Top seven days with most matches played')
fig.show()

### Plants and Defuses

In [ ]:
matches_plat_defuse = (
                        df_matches_raw.query('userId == @selected_players_list').groupby(['result'])[['plantsValue', 'defusesValue']]
                        .sum()
                        .sort_values('plantsValue', ascending = False)
                        .reset_index()
)

In [ ]:
fig = px.bar(matches_plat_defuse, x = 'result', y = 'plantsValue', text_auto='.2s',
            title="Default: various text sizes, positions and angles", color = 'result')
fig.show()

fig = px.bar(matches_plat_defuse, x = 'result', y = 'defusesValue', text_auto='.2s',
            title="Default: various text sizes, positions and angles", color = 'result')
fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Box(y=matches_plat_defuse['plantsValue'],
                    name='plantsValue'))
fig.add_trace(go.Box(y=matches_plat_defuse['defusesValue'],
                    name='defusesValue'))
fig.show()